In [ ]:
%cd ..

In [ ]:
import epych
import glob
import h5py
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import quantities as pq

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "lo_gloexp"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
def hippocampal_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l)

def visual_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("VIS" in l)

def subcortical_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l or "MB" in l or "SCi" in l or "POST" in l)

In [ ]:
NWB_FILES = glob.glob('/mnt/data/*.nwb')

In [ ]:
contrasts = {}
events = {}

In [ ]:
def common_area_title(signame, signal):
    locations = [loc.decode() for loc in signal.channels.location.values]
    prefix = os.path.commonprefix(locations)
    return "%s (%s)" % (prefix if prefix else "Subcortical", signame)

In [ ]:
for f, nwb_file in enumerate(NWB_FILES):
    basename = os.path.splitext(nwb_file)[0]
    if not os.path.exists(basename + "_go_seqctl"):
        continue
    ctrls = {
        "go_gloexp": epych.recording.Sampling.unpickle(basename + "_go_rndctl").smap(lambda sig: sig.median_filter()),
        "lo_gloexp": epych.recording.Sampling.unpickle(basename + "_lo_rndctl").smap(lambda sig: sig.median_filter()),
    }
    logging.info("Loaded random control data of %s" % nwb_file)

    for cond in CONDITIONS:
        sampling = epych.recording.Sampling.unpickle(basename + "_" + cond).smap(lambda sig: sig.median_filter())
        events[(nwb_file, cond)] = {
            "Oddball Onset": (sampling.trials[cond + "_start"].mean(), 'lightgreen'),
            "Oddball Offset": (sampling.trials[cond + "_end"].mean(), 'red'),
        }
        seqctrl = ctrls[cond]

        test_lfp, ctrl_lfp = sampling.baseline_correct(0, PRETRIAL_SECONDS), seqctrl.baseline_correct(0, PRETRIAL_SECONDS)
        contrasts[(nwb_file, cond, "lfp")] = epych.recording.trials_ttest(test_lfp, ctrl_lfp, pvalue=0.001)

        test_csd = test_lfp.smap(lambda sig: sig.downsample(4).current_source_density("vertical"))
        ctrl_csd = ctrl_lfp.smap(lambda sig: sig.downsample(4).current_source_density("vertical"))
        contrasts[(nwb_file, cond, "csd")] = epych.recording.trials_ttest(test_csd, ctrl_csd, pvalue=0.001)
        logging.info("Calculated single contrast of %s condition in %s" % (cond, nwb_file))
        del ctrl_csd, ctrl_lfp, sampling, test_csd, test_lfp
    logging.info("Calculated single contrasts of %s" % nwb_file)
    
    del ctrls

In [ ]:
for f, nwb_file in enumerate(NWB_FILES):
    basename = os.path.splitext(nwb_file)[0]
    if not os.path.exists(basename + "_go_seqctl"):
        continue

    for cond in CONDITIONS:
        erp = contrasts[(nwb_file, cond, "lfp")].erp()
        cortical = erp.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_contrast_visual_lfp.pdf" % (basename, cond), sigtitle=common_area_title, **events[(nwb_file, cond)])

        subcortical = erp.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_contrast_subcortical_lfp.pdf" % (basename, cond), sigtitle=common_area_title, **events[(nwb_file, cond)])

        csd = contrasts[(nwb_file, cond, "csd")].erp()
        cortical = csd.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_contrast_visual_csd.pdf" % (basename, cond), sigtitle=common_area_title, **events[(nwb_file, cond)])

        subcortical = csd.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_contrast_subcortical_csd.pdf" % (basename, cond), sigtitle=common_area_title, **events[(nwb_file, cond)])

        logging.info("Plotted single contrasts of %s condition in %s" % (cond, nwb_file))
    logging.info("Plotted single contrasts of %s" % nwb_file)